# WeaviateStore

[Weaviate](https://weaviate.io/) 是一个开源的向量数据库，既可以存储对象也可以存储向量，允许将向量搜索与结构化过滤结合使用。LangChain 通过 weaviate-client 包连接到 Weaviate，这是 Weaviate 的官方 Typescript 客户端。

本指南提供了使用 Weaviate [向量存储](/docs/concepts/#vectorstores) 的快速概述。如需详细了解所有 `WeaviateStore` 功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/weaviate/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`WeaviateStore`](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html) | [`@langchain/weaviate`](https://npmjs.com/@langchain/weaviate) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/weaviate?style=flat-square&label=%20&) |

## 设置

要使用 Weaviate 向量存储，你需要先设置一个 Weaviate 实例并安装 `@langchain/weaviate` 集成包。你还应该安装 `weaviate-client` 包以初始化客户端来连接你的实例，以及安装 `uuid` 包（如果你希望为索引的文档分配 ID）。

本指南还将使用 [OpenAI 嵌入模型](/docs/integrations/text_embedding/openai)，这需要你安装 `@langchain/openai` 集成包。如果需要，你也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/weaviate @langchain/core weaviate-client uuid @langchain/openai
</Npm2Yarn>
```

你需要在本地或服务器上运行 Weaviate。更多信息请参阅 [Weaviate 官方文档](https://weaviate.io/developers/weaviate/installation)。

### 凭据

设置好实例后，请设置以下环境变量：

```typescript
// 如果在本地运行，包含端口号，例如 "localhost:8080"
process.env.WEAVIATE_URL = "YOUR_WEAVIATE_URL";
// 可选，用于云部署
process.env.WEAVIATE_API_KEY = "YOUR_API_KEY";
```

如果你在本指南中使用 OpenAI 嵌入模型，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动追踪，也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

### 连接一个 Weaviate 客户端

在大多数情况下，你应该使用以下连接辅助函数之一来连接到你的 Weaviate 实例：

- connectToWeaviateCloud
- connectToLocal
- connectToCustom

In [ ]:
import { WeaviateStore } from "@langchain/weaviate";
import { OpenAIEmbeddings } from "@langchain/openai";
import weaviate from "weaviate-client";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const weaviateClient = weaviate.connectToWeaviateCloud({
   clusterURL: process.env.WEAVIATE_URL!, 
	 options : {
      authCredentials: new weaviate.ApiKey(process.env.WEAVIATE_API_KEY || ""),
      headers: {
        "X-OpenAI-Api-Key": process.env.OPENAI_API_KEY || "",
        "X-Cohere-Api-Key": process.env.COHERE_API_KEY || "",
      },
    },
});

### 初始化向量存储
要创建集合，请至少指定集合名称。如果不指定任何属性，`auto-schema` 会自动创建它们。

In [ ]:
const vectorStore = new WeaviateStore(embeddings, {
  client: weaviateClient, 
  // Must start with a capital letter
  indexName: "Langchainjs_test",
});

要使用 Weaviate 的命名向量、向量化器、重排序器、生成模型等，请在启用向量存储时使用 `schema` 属性。创建向量存储时，`schema` 中的集合名称和其他属性将具有优先权。

In [ ]:
const vectorStore = new WeaviateStore(embeddings, {
  client: weaviateClient, 
  schema: {
    name: "Langchainjs_test",
    description: "A simple dataset",
    properties: [
      {
        name: "title",
        dataType: dataType.TEXT,
      },
      {
        name: "foo",
        dataType: dataType.TEXT,
      },
    ],
    vectorizers: [
      vectorizer.text2VecOpenAI({
        name: "title",
        sourceProperties: ["title"], // (Optional) Set the source property(ies)
        // vectorIndexConfig: configure.vectorIndex.hnsw()   // (Optional) Set the vector index configuration
      }),
    ],
    generative: weaviate.configure.generative.openAI(),
    reranker: weaviate.configure.reranker.cohere(),
  },
});

## 管理向量存储

### 向向量存储中添加项目

**注意：** 如果您希望将ID与索引的文档相关联，则这些ID必须是UUID。

In [2]:
import type { Document } from "@langchain/core/documents";
import { v4 as uuidv4 } from "uuid";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];
const uuids = [uuidv4(), uuidv4(), uuidv4(), uuidv4()];

await vectorStore.addDocuments(documents, { ids: uuids });

[
  '610f9b92-9bee-473f-a4db-8f2ca6e3442d',
  '995160fa-441e-41a0-b476-cf3785518a0d',
  '0cdbe6d4-0df8-4f99-9b67-184009fee9a2',
  '18a8211c-0649-467b-a7c5-50ebb4b9ca9d'
]


### 从向量存储中删除项目

你可以通过传递 `filter` 参数按 ID 进行删除：

In [3]:
await vectorStore.delete({ ids: [uuids[3]] });

## 查询向量存储

一旦你的向量存储创建完成并将相关文档添加进去后，在运行链或代理时很可能需要对其进行查询。
在 Weaviate v3 中，客户端主要通过与 `collections` 交互来操作数据库中的对象。`collection` 对象可以在整个代码库中重复使用。
### 直接查询

可以如下进行简单的相似性搜索。`Filter` 辅助类使使用带有条件的过滤器变得更加简单。v3 客户端简化了 `Filter` 的使用方式，使你的代码更加简洁清晰。

关于 Weaviate 过滤器语法的更多信息，请参见 [此页面](https://weaviate.io/developers/weaviate/api/graphql/filters)。

In [ ]:
import { Filters } from "weaviate-client";

const collection = client.collections.use('Langchainjs_test');

const filter = Filters.and(collection.filter.byProperty("source").equal("https://example.com"))

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


如果你想执行相似性搜索并获得相应的分数，可以运行以下命令：

In [15]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.835] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.852] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 混合搜索
在 Weaviate 中，`混合搜索` 通过融合向量搜索和关键词（BM25F）搜索的两个结果集来组合结果。要更改关键词和向量组件的相对权重，请在查询中设置 `alpha` 值。

查看 __[文档](https://weaviate.io/developers/weaviate/search/hybrid)__ 以获取完整的混合搜索选项列表。

In [ ]:
const results = await vectorStore.hybridSearch("biology", 
  {
    limit: 1,
    alpha: 0.25,
    targetVector: ["title"],
    rerank: {
      property: "title",
      query: "greeting",
    },
});

### 检索增强生成（RAG）
检索增强生成（RAG）将信息检索与生成式AI模型相结合。

在 Weaviate 中，RAG 查询包含两个部分：一个搜索查询和一个提供给模型的提示。Weaviate 首先执行搜索，然后将搜索结果和你的提示一起传递给生成式AI模型，最后返回生成的响应。
   * @param query 要搜索的查询。
   * @param options 执行混合搜索的可用选项
   * @param generate 生成的可用选项。请查看文档以获取完整列表

In [ ]:
const results = await vectorStore.generate("hello world",
    {
        singlePrompt: {
            prompt: "Translate this into German: {title}",
        },
        config: generativeParameters.openAI({
            model: "gpt-3.5-turbo",
        }),
    },
    {
        limit: 2,
        targetVector: ["title"],
    }
);

### 通过转换为检索器进行查询

您还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在您的链中更方便地使用。

In [16]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何操作：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考文档

如需详细了解所有 `WeaviateStore` 功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html)。